In [54]:
import sys
sys.path.append('..')

import os
from sqlalchemy import create_engine, text
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
from shapely import wkb
from geopandas.tools import sjoin

from utilities_amigocloud import AmigocloudFunctions

In [55]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
PATH_CAT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\catastro_S09_MIERCOLES.shp'
PATH_XLSX_GRUPOS = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'

In [56]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_planificacion():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_pulverizacion.planificacion_pulv
            WHERE procesado=false;
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def marcar_como_procesado(id_os):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.planificacion_pulv
                SET procesado = true
                WHERE id = :id_os
            """)
            conn.execute(query, {"id_os": id_os})
            print(f"✔️ id {id_os} marcado como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None

def obtener_parte_diario_por_os(ors):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_pulverizacion.parte_diario_pulv where os = {ors}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def convertir_a_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    return geometry

def convertir_a_wkb(polygon):
    wkb_data = wkb.dumps(polygon, hex=True)
    return wkb_data

def cargar_a_amigocloud(gdf):
    # repreyectar a WGS84
    gdf_pla_gral = gdf.to_crs(epsg=4326)
    # convertir poligonos a multipoligonos
    gdf_pla_gral['geom'] = gdf_pla_gral['geom'].apply(convertir_a_multipolygon)
    
    gdf_pla_gral['unidad_01'] = gdf_pla_gral['unidad_01'].astype(int)
    gdf_pla_gral['unidad_03'] = gdf_pla_gral['unidad_03'].astype(int)
    gdf_pla_gral['os'] = gdf_pla_gral['os'].astype(int)
    gdf_pla_gral['soca'] = gdf_pla_gral['soca'].astype(int)
    gdf_pla_gral['id'] = gdf_pla_gral['id'].astype(int)
    gdf_pla_gral['inst'] = gdf_pla_gral['inst'].astype(int)
    gdf_pla_gral['mezcla'] = gdf_pla_gral['mezcla'].astype(int)
    
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 35248
    for index, row in gdf_pla_gral.iterrows():
        wkb_hex = convertir_a_wkb(row['geom'])
        insert_sql = f"""
            INSERT INTO dataset_360912 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, mezcla, geometry)
            VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['os']}', '{row['mezcla']}', ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
        """
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        print(f'Lote registrado en AmigoCloud')
    return None

In [57]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

In [58]:
gdf_cat = gpd.read_file(PATH_CAT)
gdf_plan = obtener_planificacion()
gdf_plan

,id,geom,codigo_canero,nombre_canero,procesado,mezcla,obs
0,59,"MULTIPOINT (480429.556 8080943.467, 480607.805...",2816,DONOSO BLANCO ALBERTO,False,50,None


# CARGAR PARTE DIARIO

In [59]:
gdf_intersect_result = gpd.sjoin(gdf_cat, gdf_plan, how='inner')
os_para_cargar = list(set(gdf_intersect_result['id_right']))
os_para_cargar

[59]

In [60]:
gdf_intersect_result.head(3)['mezcla']

243     50
246     50
1772    50
Name: mezcla, dtype: int64

In [61]:
gdf_result = gdf_intersect_result[['geometry', 'unidad_01', 'unidad_02', 'codigo_canero', 'nombre_canero', 'unidad_05', 'area', 'id_right', 'soca', 'zona', 'mezcla']].copy()
gdf_result['area'] = gdf_result.geometry.area / 10000
gdf_result.rename(columns={
    'geometry': 'geom',
    'codigo_canero': 'unidad_03',
    'nombre_canero': 'unidad_04',
    'id_right': 'os',
    'zona': 'inst'
}, inplace=True)

gdf_result = gdf_result.set_geometry("geom")
gdf_result['unidad_01'] = gdf_result['unidad_01'].astype(int)
gdf_result['unidad_03'] = gdf_result['unidad_03'].astype(int)
gdf_result['os'] = gdf_result['os'].astype(int)
gdf_result['soca'] = gdf_result['soca'].astype(int)
gdf_result['inst'] = gdf_result['inst'].astype(int)
gdf_result['mezcla'] = gdf_result['mezcla'].astype(int)

In [62]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst,mezcla
243,"POLYGON ((481748.566 8079592.177, 481741.595 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,C3,10.248108,59,0,69,50
246,"POLYGON ((481047.643 8080996.177, 481087.784 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,C1,7.012051,59,0,69,50
1772,"POLYGON ((480712.080 8080887.638, 480573.065 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,B1.3,4.658385,59,0,69,50
7668,"POLYGON ((480712.080 8080887.638, 480656.325 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,B1.1,1.970184,59,0,69,50
7694,"POLYGON ((481119.930 8080378.683, 480947.024 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,B2,15.928836,59,0,69,50
11292,"POLYGON ((480940.160 8079892.582, 480864.373 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,A3.2,3.070818,59,15,69,50
11293,"POLYGON ((482403.769 8078490.606, 482402.999 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,A12,5.821546,59,5,69,50
11299,"POLYGON ((480233.501 8081012.800, 480241.372 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,A1,11.461584,59,1,69,50
11304,"POLYGON ((480934.628 8080276.441, 480709.149 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,A2.3,9.504411,59,3,69,50
11306,"POLYGON ((481509.557 8080055.714, 481476.943 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,B3,19.349682,59,6,69,50


In [63]:
# GARANTIZAR LA INST DEL CAÑERO
# cargar grupos de cosecha
df_grupos = pd.read_excel(PATH_XLSX_GRUPOS, sheet_name='CODIGOS')
df_grupos = df_grupos[df_grupos['INSTITUCION'].notna()]
df_grupos['CODIGO CAÑERO'] = df_grupos['CODIGO CAÑERO'].astype(int)
df_grupos['INSTITUCION'] = df_grupos['INSTITUCION'].astype(int)

# Crear un diccionario de mapeo: {codigo_cañero: institucion}
mapa_institucion = df_grupos.set_index('CODIGO CAÑERO')['INSTITUCION'].to_dict()
# Reemplazar los valores de 'inst' en gdf_result usando el diccionario
# a partir del diccionario, busca el codigo cañero (unidad_03), y reempalza la institucion
gdf_result['inst'] = gdf_result['unidad_03'].map(mapa_institucion)

In [64]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst,mezcla
243,"POLYGON ((481748.566 8079592.177, 481741.595 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,C3,10.248108,59,0,69,50
246,"POLYGON ((481047.643 8080996.177, 481087.784 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,C1,7.012051,59,0,69,50
1772,"POLYGON ((480712.080 8080887.638, 480573.065 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,B1.3,4.658385,59,0,69,50
7668,"POLYGON ((480712.080 8080887.638, 480656.325 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,B1.1,1.970184,59,0,69,50
7694,"POLYGON ((481119.930 8080378.683, 480947.024 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,B2,15.928836,59,0,69,50
11292,"POLYGON ((480940.160 8079892.582, 480864.373 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,A3.2,3.070818,59,15,69,50
11293,"POLYGON ((482403.769 8078490.606, 482402.999 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,A12,5.821546,59,5,69,50
11299,"POLYGON ((480233.501 8081012.800, 480241.372 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,A1,11.461584,59,1,69,50
11304,"POLYGON ((480934.628 8080276.441, 480709.149 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,A2.3,9.504411,59,3,69,50
11306,"POLYGON ((481509.557 8080055.714, 481476.943 8...",74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,B3,19.349682,59,6,69,50


In [65]:
# agregar nuevos registros a la base de datos
gdf_result.to_postgis("parte_diario_pulv", obtener_engine(), schema="drones_pulverizacion", if_exists="append")

In [66]:
os_para_cargar

[59]

In [67]:
# CARGAR A AMIGOCLOD
for i in os_para_cargar:
    plan_os = obtener_parte_diario_por_os(i)
    cargar_a_amigocloud(plan_os)
    marcar_como_procesado(i)

Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 59 marcado como procesado.
